# Langchain with OpenAI
This notebook explores the [langchain framework](https://langchain.readthedocs.io/en/latest/index.html) that helps develop with large language models for various tasks including [question answering](https://langchain.readthedocs.io/en/latest/modules/indexes/chain_examples/question_answering.html). Here, we are using the open-ai api as the choice of llm. You will need to provide the api key as the `OPENAI_API_KEY` environment variable to execute this notebook. For the dataset, we use public Red Hat OpenShift on AWS (ROSA) docs.

In [ ]:
!pip install langchain openai chromadb unstructured evaluate bert_score

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import MarkdownTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from dotenv import load_dotenv, find_dotenv
import os

In [2]:
load_dotenv(find_dotenv("credentials.env"), override=True)

True

## Semantic search within the documents
This step can be seen as retriever for the QA model. This performs a similar function as the dense passage retrieval and term frequency based information retrieval tasks. If we have a large collection of documents, this step filters the relevant parts for the text generators to use as input. Here, we are using [chromadb](https://www.trychroma.com/) as our choice of vectorstore following the [langchain QA tutorial](https://langchain.readthedocs.io/en/latest/modules/indexes/chain_examples/question_answering.html). 

In [3]:
loader = TextLoader('../data/external/rosaworkshop/14-faq.md')
documents = loader.load()
text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [4]:
query = "What is Red Hat OpenShift Service on AWS (ROSA)?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

General

### What is Red Hat OpenShift Service on AWS (ROSA)?
Red Hat Openshift Service on AWS (ROSA) is a fully-managed turnkey application platform that allows you to focus on what matters most, delivering value to your customers by building and deploying applications. Red Hat SRE experts manage the underlying platform so you don’t have to worry about the complexity of infrastructure management.

### Where can I go to get more information/details?
- [ROSA Webpage](https://www.openshift.com/products/amazon-openshift)
- [ROSA Workshop](https://www.rosaworkshop.io)
- [ROSA Documentation](https://docs.openshift.com/rosa/welcome/index.html)


## Question answering 

For question answering, we use the llm chain feature of the langchain framework.

In [5]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")

In [6]:
query = "What is Red Hat OpenShift Service on AWS (ROSA)?"
chain.run(input_documents=docs, question=query)

' Red Hat OpenShift Service on AWS (ROSA) is a fully-managed turnkey application platform that allows you to focus on what matters most, delivering value to your customers by building and deploying applications. Red Hat SRE experts manage the underlying platform so you don’t have to worry about the complexity of infrastructure management.'

## Question answering outputs with sources

In [7]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Red Hat OpenShift Service on AWS (ROSA) is a fully-managed turnkey application platform that allows you to focus on what matters most, delivering value to your customers by building and deploying applications. It includes container management, automation (Operators), networking, load balancing, service mesh, CI/CD, firewall, monitoring, registry, authentication, and authorization capabilities. The underlying node OS used is Red Hat Enterprise Linux CoreOS (RHCOS).\nSOURCES: ../data/external/rosaworkshop/14-faq.md'}

## Loading all the ROSA documents

In [8]:
loader = DirectoryLoader('../data/external', glob="**/*.md", loader_cls=TextLoader)
documents = loader.load()

In [9]:
## To do: better split of docs
text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()

In [10]:
docsearch = Chroma.from_documents(texts, embeddings, persist_directory='../data/interim')
#docsearch.persist()

Running Chroma using direct local API.
No existing DB found in ../data/interim, skipping load
No existing DB found in ../data/interim, skipping load
Exiting: Cleaning up .chroma directory
Persisting DB to disk, putting it in the save folder ../data/interim


In [11]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

## Next, we are going to answer example questions and look at the model responses

In [12]:
def answer_question(query, index, chain):
    """
    Takes in query, index to search from, and llm chain to generate answer
    """
    ## Retrieve docs
    docs = index.similarity_search(query)
    ## Generate answer
    answer = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
    return answer['output_text']

In [14]:
queries = ['Where can I see a roadmap or make feature requests for the service?',
           'How is the pricing of Red Hat OpenShift Service on AWS calculated?',
           'Is there an upfront commitment?',
           'How can I delete ROSA cluster?',
           'How can I automatically deploy ROSA cluster?',
           'How can my ROSA cluster autoscale?',
           'How can I install aws load balancer controller',
           'How can I install Prometheus Operator with my ROSA cluster?',
           'What time is it?',
           'How can I federate metrics to a centralized Prometheus Cluster?',
           'What is the meaning of life?']

for query in queries:
    answer = answer_question(query, docsearch, chain)
    print(f'Query: {query} \n Answer: {answer}')

Query: Where can I see a roadmap or make feature requests for the service? 
 Answer:  You can submit a Jira issue for the most relevant documentation component to make feature requests for the service.
SOURCES: ../data/external/rosa-docs/service_mesh.md
Query: How is the pricing of Red Hat OpenShift Service on AWS calculated? 
 Answer:  Red Hat OpenShift Service on AWS has three components to its cost: an hourly cluster fee, pricing per worker node, and underlying AWS infrastructure costs. The hourly cluster fee is $0.03/cluster/hour, and the pricing per worker node is $0.171 per 4vCPU/hour for on-demand consumption, $0.114 per 4vCPU/hour for a 1-year RI commitment, and $0.076 per 4vCPU/hour for a 3-year RI commitment.
SOURCES: ../data/external/rosaworkshop/14-faq.md
Query: Is there an upfront commitment? 
 Answer:  There is no required upfront commitment.
SOURCES: ../data/external/rosaworkshop/14-faq.md
Query: How can I delete ROSA cluster? 
 Answer:  To delete a ROSA cluster, use the

## Next Steps

* Understand and experiment with different types of [chains](https://langchain.readthedocs.io/en/latest/modules/chains/combine_docs.html) and update the `chain_type` parameter.
* Explore Hyde and other embedding sources.
* Explore Bloom and custom llm with langchain.